In [19]:
from pathlib import Path
import pandas as pd
import codecs
from typing import List, Set

In [20]:
import re
import math
import codecs
import random
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors
import editdistance
from collections import Counter
from collections import defaultdict
import time
from functools import lru_cache

### See Data, Feel Data, Preprocess Data [1 балл]

In [23]:
words_path = Path("/content/words2.txt")
broken_texts_path = Path("/content/broken_texts.csv")
correct_texts_path = Path("/content/derived.csv")

In [53]:
df = pd.read_csv(broken_texts_path)
test_df = pd.read_csv(correct_texts_path)

In [25]:
df.head()

,id,text
0,0,не обнаруживается различий в общем объеме серо...
1,1,"интегумент ( от - покрывало , покров ) - терми..."
2,2,"22 июня 2013 года решениме большинстав судей ,..."
3,3,в 1987 mi выпустила альбом all the best !
4,4,путевая машинная станция 79 была создана 16 фе...


In [57]:
texts = df["text"].tolist()
correct_texts = test_df["text"].tolist()

In [61]:
for text, correct_text in zip(texts, correct_texts):
    if "  " in text:
        print(text)
        print(correct_text)
        print('\n')
        break

; песни и романсы х . ф . ахметова , з .  .. исмагилова , т . ш . каримова , с . а . низаметдинова ,, н . г . сабитова , р . х . сахаутдиновой , а  б . туктагулова и др  народная артистка рб ( 2012 ) .
; песни и романсы х . ф . ахметова , з . г . исмагилова , т . ш . каримова , с . а . низаметдинова , н . г . сабитова , р . х . сахаутдиновой , а . б . туктагулова и др . народная артистка рб ( 2012 ) .




In [62]:
cnt = 0
for text, correct_text in zip(texts, correct_texts):
    if len(text.split()) != len(correct_text.split()):
        cnt += 1
cnt

4485

Видно, что очень много где пропущены однобуквенные слова, поэтому как раз и нельзя делать просто `split()` - из-за него всё съезжает

In [27]:
with open(words_path, 'r') as f:
  words_list = sorted(set(word.strip() for word in f.readlines()))

In [28]:
assert words_list[::20000] == ['!',
                               'daewoo',
                               'агенобарб',
                               'биотопа',
                               'восхитительный',
                               'дебальцево',
                               'зафрахтованные',
                               'километровой',
                               'лесными',
                               'московщина',
                               'овдовевшего',
                               'пионер-11',
                               'провозглашавшая',
                               'рыжиков',
                               'спокоен',
                               'тэйна',
                               'хэй']

In [29]:
assert len(words_list) == 339275

In [30]:
def read_data(df: pd.DataFrame) -> str:
    text = '\n'.join(df["text"].tolist())

## Abstract it a little bit

In [31]:
from abc import abstractmethod

class Speller:
    
    @abstractmethod
    def fit(self, words: List[str], **kwargs):
        raise NotImplementedError()
    
    @abstractmethod
    def predict(self, data: str, **kwargs) -> str:
        raise NotImplementedError()

In [32]:
class BaseRectifier:
    
    @abstractmethod
    def rectify(self, data: List[str]) -> List[str]:
        raise NotImplementedError

In [33]:
def mesure_metric(gt: List[str], broken: List[str], corrected: List[str]) -> float:
    all_tokens = 0
    correct = 0
    for gt_sentence, broken_sentence, corrected_sentence in zip(gt, broken, corrected):
        for gt_token, broken_token, corrected_token in zip(gt_sentence.split(), broken_sentence.split(), corrected_sentence.split()):
            if gt_token != broken_token:
                all_tokens += 1
                if gt_token == corrected_token:
                    correct += 1
    return correct / all_tokens

## Rectifier [1 балл]

In [34]:
from tqdm.notebook import tqdm

In [36]:
class Rectifier(BaseRectifier):
    
    def __init__(self, words: Set[str], speller: Speller):
        self._words = words
        self._speller = speller
    
    def rectify(self, data: List[str]) -> List[str]:
        corrected = []
        for sentence in tqdm(data):
            corrected.append(' '.join([
                self._speller.predict(word) if word not in self._words else word for word in sentence.split(" ")
            ]))

        return corrected

## Simple Speller [2 балла]

In [83]:
class SimpleSpeller(Speller):
    "Inspired by Petr Norvig Spellchecker"
    
    def __init__(self):
        self._letters = 'абвгдеёжзиклмнопрстуфхцчшщъыьэюя'
    
    def fit(self, words: List[str], **kwargs):
        self._words = set(words)
        return self
    
    @lru_cache(maxsize=10000)
    def predict(self, word: str) -> str:
        suggests = self._get_candidates(word)
        
        closest_word = word if len(word) > 0 else "."
        minimal_distance = 1000
        cnt = 1
        for suggest in suggests:
            d = editdistance.eval(word, suggest)
            if d < minimal_distance:
                closest_word = suggest
                minimal_distance = d
                cnt = 1
            elif d == minimal_distance:
                cnt += 1
        # print(closest_word, cnt)
        return closest_word
    
    def _get_candidates(self, word: str) -> Set[str]:
        candidates = set([w for w in self._get_edits(word) if w in self._words and len(w) > 0])
        return candidates
    
    def _get_edits(self, word: str) -> Set[str]:
        splits     = [(word[:i], word[i:]) for i in range(1, len(word) - 1)]
        deletes    = [word[:i] + word[i + 1:] for i in range(len(word))]
        transposes = [word[:i] + word[i + 1] + word[i] + word[i + 2:] for i in range(len(word) - 1)]
        replaces   = [word[:i] + letter + word[i + 1:] for i in range(len(word)) for letter in self._letters]
        inserts    = [part[0] + letter + part[1] for part in splits for letter in self._letters]
        return set(deletes + transposes + replaces + inserts)

In [84]:
rectifier = Rectifier(set(words_list), SimpleSpeller().fit(words_list))

In [85]:
corrected_text = rectifier.rectify(texts)

  0%|          | 0/20000 [00:00<?, ?it/s]

In [86]:
gt = pd.read_csv(correct_texts_path)["text"]

In [87]:
assert mesure_metric(gt, texts, corrected_text) > 0.18

In [88]:
mesure_metric(gt, texts, corrected_text)

0.767358803986711

## Intermediate Speller [2 балла]

In [45]:
def wrap(word):
    return u' %s ' % word

In [46]:
class IntermediateSpeller(Speller):
    
    def __init__(self):
        self.neighbours = 5
        self.vectorizer = CountVectorizer(analyzer="char", ngram_range=(1, 2), binary=True)
        self.searcher = NearestNeighbors(algorithm="auto", n_neighbors=self.neighbours, radius=3)
    
    def fit(self, words: List[str], **kwargs):
        self.words = np.array(words)
        encoded_words = self.vectorizer.fit_transform(self.words).tocoo()
        self.searcher.fit(encoded_words)
        return self
    
    @lru_cache(maxsize=10000)
    def predict(self, word: str) -> str:
        wrapped_word = u" %s " % word
        word_vec = self.vectorizer.transform([wrapped_word])

        closest_word = word
        minimal_distance = 1000

        dists, idxs = self.searcher.kneighbors(word_vec)
        for d, idx in zip(dists[0], idxs[0]):
            suggested_word = self.words[idx]
            if d < minimal_distance:
                closest_word = suggested_word.strip()
                minimal_distance = d
        return closest_word

In [47]:
wrapped_words_list = list(map(lambda x : wrap(x), words_list))
rectifier = Rectifier(set(words_list), IntermediateSpeller().fit(wrapped_words_list))

In [48]:
corrected_text = rectifier.rectify(texts)

  0%|          | 0/20000 [00:00<?, ?it/s]

In [49]:
gt = pd.read_csv(correct_texts_path)["text"]

In [50]:
assert mesure_metric(gt, texts, corrected_text) > 0.2

In [51]:
mesure_metric(gt, texts, corrected_text)

0.7997626957759848

## Your Good Speller [2 балла]

Сейчас я попробую обучить языковую модель на словах.  
Много где есть несколько вариантов слов с одинаковым редакционным расстоянием. И в *SimpleSpeller* я просто брал первое из слов. Почему бы не взять самое "адекватное", согласно языковой модели?

In [101]:
class LanguageModel(object):
    """
    Реализуйте n-граммную модель со сглаживанием Лапласа
    """

    def __init__(self, ngram_size=2, alpha=1):

        if ngram_size < 2:
            raise Exception

        self.ngram_size = ngram_size

        self.ngram_vectorizer = CountVectorizer(analyzer="char", ngram_range=(ngram_size, ngram_size))
        self.context_vectorizer = CountVectorizer(analyzer="char", ngram_range=(ngram_size - 1, ngram_size - 1))
        self.ngram_counts = defaultdict(int)
        self.context_counts = defaultdict(int)
        self.ngrams_set_size = None
        self.wrap = lambda x: "^" * (self.ngram_size - 1) + x + "$" * (self.ngram_size - 1)
        self.alpha = alpha

    def fit(self, words):
        """
        Обучение модели на тексте, разбитом на предложения
        :param sentences: список предложений
        """

        print("Fitting words")
        wrapped_words = list(map(self.wrap, words))

        counts_matrix = self.ngram_vectorizer.fit_transform(wrapped_words)
        counts_context_matrix = self.context_vectorizer.fit_transform(wrapped_words)
        self.ngrams_set_size = len(self.ngram_vectorizer.vocabulary_)

        print("Summing...")

        sum_ngram = np.asarray(counts_matrix.sum(axis=0)).flatten()
        sum_context = np.asarray(counts_context_matrix.sum(axis=0)).flatten()

        print("shapes: ", sum_ngram.shape, sum_context.shape)
        print("Iterating through ngrams...")

        for ngram in self.ngram_vectorizer.vocabulary_:
            self.ngram_counts[ngram] = sum_ngram[self.ngram_vectorizer.vocabulary_[ngram]]
        for ngram in self.context_vectorizer.vocabulary_:
            self.context_counts[ngram] = sum_context[self.context_vectorizer.vocabulary_[ngram]]

        return self

    def perplexity(self, words):
        wrapped_words = list(map(self.wrap, words))
        lens = np.array([len(word) for word in wrapped_words])
        logprobs = np.array(self.log_prob(wrapped_words)) / lens
        return (1 / 2 ** logprobs).mean()

    def perplexitys(self, words):
        wrapped_words = list(map(self.wrap, words))
        lens = np.array([len(word) for word in wrapped_words])
        logprobs = np.array(self.log_prob(wrapped_words)) / lens
        return np.exp2(-logprobs)

    def __log_prob_word(self, word):
        """
        Оцениваем логарифм вероятности предложения частотами в рамках данной модели
        :param word:
        :return: логарифм вероятности
        """
        n = self.ngram_size
        V = self.ngrams_set_size
        probs = []
        for i in range(len(word) - n):
            probs.append((self.ngram_counts[word[i: i + n]] + self.alpha) / (self.context_counts[word[i: i + n - 1]] + self.alpha * V))
        return np.log2(probs).sum()

    def log_prob(self, words_list):
        """
        Логарифм вероятности каждого из предложений в списке
        """
        return list(map(self.__log_prob_word, words_list))

In [102]:
class NotBadSpeller(Speller):
    "Inspired by Petr Norvig Spellchecker"
    
    def __init__(self, ngrams=2, alpha=0.5):
        self._letters = 'абвгдеёжзиклмнопрстуфхцчшщъыьэюя'
        self.ngrams_size = ngrams
        self.alpha = alpha
        self._lm = None
    
    def fit(self, words: List[str], **kwargs):
        self._words = set(words)
        self._lm = LanguageModel(self.ngrams_size, self.alpha).fit(words)
        return self
    
    @lru_cache(maxsize=10000)
    def predict(self, word: str) -> str:
        suggests = list(self._get_candidates(word))

        closest_words = [word]
        minimal_distance = 1000

        for suggest in suggests:
            d = editdistance.eval(word, suggest)
            if d < minimal_distance:
                closest_word = suggest
                minimal_distance = d
                closest_words = [suggest]
            elif d == minimal_distance:
                closest_words.append(suggest)

        perplexitys = self._lm.perplexitys(closest_words)
        closest_word = word
        best_perp = 1e6
        for suggest, perp in zip(closest_words, perplexitys):
            if perp < best_perp:
                best_perp, closest_word = perp, suggest

        return closest_word
    
    def _get_candidates(self, word: str) -> Set[str]:
        return set([w for w in self._get_edits(word) if w in self._words and len(w) > 0])
    
    def _get_edits(self, word: str) -> Set[str]:
        splits     = [(word[:i], word[i:]) for i in range(len(word) + 1)]
        deletes    = [word[:i] + word[i + 1:] for i in range(len(word))]
        transposes = [word[:i] + word[i + 1] + word[i] + word[i + 2:] for i in range(len(word) - 1)]
        replaces   = [word[:i] + letter + word[i + 1:] for i in range(len(word)) for letter in self._letters]
        inserts    = [part[0] + letter + part[1] for part in splits for letter in self._letters]
        return set(deletes + transposes + replaces + inserts)

In [110]:
rectifier = Rectifier(set(words_list), NotBadSpeller(ngrams=3, alpha=0.75).fit(words_list))

Fitting words
Summing...
shapes:  (38007,) (3274,)
Iterating through ngrams...


In [111]:
corrected_text = rectifier.rectify(texts)

  0%|          | 0/20000 [00:00<?, ?it/s]

In [112]:
mesure_metric(gt, texts, corrected_text)

0.7843735168485999

In [113]:
assert mesure_metric(gt, texts, corrected_text) > 0.4

Ну да, стало чуточку лучше

## Your Amazing Speller [2 балла]

Попробуем всё ту же идею, что и в самом простом *SpellChecker* - даже без языковой модели, так как явного выигрыша она не даёт.  
Вместо будем допускать большее количество опечаток в одном слове.  
Также попробуем учесть, что *replace* - то есть какая-то не так буква - это обычно сосед на клавиатуре. Бывают, конечно, разные раскладки, но предположим, что qwerty  
И ещё одна вещь до сих пор была странной. Мы делаем *_get_edits*, а потом сравниваем по расстоянию Левенштейна. Хочется, чтобы расстояние учитывало ещё и перестановки соседних букв. Звучит как расстояние Дамерау-Левенштейна

In [114]:
from nltk.metrics.distance import edit_distance

In [126]:
class MaybeGoodSpeller(Speller):
    "Inspired by Petr Norvig Spellchecker"
    
    def __init__(self, ngrams=2, alpha=0.5):
        self._letters = 'абвгдеёжзиклмнопрстуфхцчшщъыьэюя'
        self._letters_dict = {
            'а': ['к', 'е', 'в', 'п', 'с', 'м'],
            'б': ['ь', 'л', 'д', 'ю'],
            'в': ['у', 'к', 'ы', 'а', 'ч', 'ч', 'с'],
            'г': ['ш', 'р', 'о'],
            'д': ['щ', 'з', 'л', 'ж', 'б', 'ю'],
            'е': ['к', 'н', 'а', 'п', 'ё'],
            'ё': ['е', 'к', 'а', 'п', 'р', 'н'],
            'ж': ['з', 'х', 'д', 'э', 'ю'],
            'з': ['щ', 'д', 'ж', 'э', 'х'],
            'и': ['м', 'п', 'р', 'т'],
            'й': ['ф', 'ы', 'ц'],
            'к': ['у', 'в', 'а', 'п', 'е'],
            'л': ['о', 'ш', 'щ', 'д', 'ь', 'б'],
            'м': ['с', 'а', 'п', 'и'],
            'н': ['е', 'п', 'р', 'о', 'г'],
            'о': ['р', 'г', 'ш', 'л', 'т', 'ь'],
            'п': ['а', 'е', 'н', 'р', 'м', 'и'],
            'р': ['п', 'и', 'т', 'о', 'г', 'н'],
            'с': ['ч', 'в', 'а', 'м'],
            'т': ['и', 'р', 'о', 'ь'],
            'у': ['ц', 'ы', 'в', 'а', 'к'],
            'ф': ['й', 'ц', 'ы', 'я'],
            'х': ['з', 'ж', 'э', 'ъ'],
            'ц': ['й', 'ф', 'ы', 'в', 'у'],
            'ч': ['я', 'ы', 'в', 'с'],
            'ш': ['г', 'о', 'л', 'д', 'щ'],
            'щ': ['ш', 'л', 'д', 'ж', 'з'],
            'ъ': ['х', 'э'],
            'ы': ['ф', 'ц', 'у', 'в', 'я', 'ч'],
            'ь': ['т', 'о', 'л', 'б'],
            'э': ['ю', 'ж', 'х'],
            'ю': ['б', 'д', 'ж'],
            'я': ['ф', 'ы', 'ч'],
            '.': ['ю', 'ж', 'э'],
            ',': ['ю', 'ж', 'э'],
            '1': ['ё', 'й', 'ц'],
            '2': ['й', 'ц'],
            '3': ['ц', 'у'],
            '4': ['у', 'к'],
            '5': ['к', 'е'],
            '6': ['е', 'н'],
            '7': ['н', 'г'],
            '8': ['г', 'ш'],
            '9': ['ш', 'щ'],
            '0': ['щ', 'з'],
            '-': ['з', 'х'],
            '=': ['х', 'ъ'],
            '\\': ['ъ']
        }
        self.ngrams_size = ngrams
        self.alpha = alpha
        self._lm = None
    
    def fit(self, words: List[str], **kwargs):
        self._words = set(words)
        self._lm = LanguageModel(self.ngrams_size, self.alpha).fit(words)
        return self
    
    @lru_cache(maxsize=10000)
    def predict(self, word: str) -> str:
        suggests = list(self._get_candidates(word))

        closest_words = [word]
        minimal_distance = 1000

        for suggest in suggests:
            d = edit_distance(word, suggest, transpositions=True)
            if d < minimal_distance:
                closest_word = suggest
                minimal_distance = d
                closest_words = [suggest]
            elif d == minimal_distance:
                closest_words.append(suggest)

        perplexitys = self._lm.perplexitys(closest_words)
        closest_word = word
        best_perp = 1e6
        for suggest, perp in zip(closest_words, perplexitys):
            if perp < best_perp:
                best_perp, closest_word = perp, suggest

        return closest_word
    
    def _get_candidates(self, word: str) -> Set[str]:
        candidates = set()
        for word_ in self._get_edits(word, neighbors_only=True):
            candidates_ = set(self._get_edits(word_, neighbors_only=True, return_inserts=False)) | set([word_])
            candidates |= set([w for w in candidates_ if w in self._words])
        return candidates
    
    def _get_edits(self, word: str, neighbors_only=False, return_inserts=True) -> Set[str]:
        splits     = [(word[:i], word[i:]) for i in range(len(word) + 1)]
        deletes    = [word[:i] + word[i + 1:] for i in range(len(word))]
        transposes = [word[:i] + word[i + 1] + word[i] + word[i + 2:] for i in range(len(word) - 1)]
        replaces   = [word[:i] + letter + word[i + 1:] for i in range(len(word)) for letter in (
            self._letters_dict[word[i]] if neighbors_only and word[i] in self._letters_dict else self._letters
            )]
        inserts    = [] if not return_inserts else [part[0] + letter + part[1] for part in splits for letter in self._letters]
        return set(deletes + transposes + replaces + inserts)

In [128]:
rectifier = Rectifier(set(words_list), MaybeGoodSpeller(ngrams=3, alpha=0.75).fit(words_list))

Fitting words
Summing...
shapes:  (38007,) (3274,)
Iterating through ngrams...


In [129]:
corrected_text = rectifier.rectify(texts)

  0%|          | 0/20000 [00:00<?, ?it/s]

In [130]:
mesure_metric(gt, texts, corrected_text)

0.792797816801139

In [131]:
assert mesure_metric(gt, texts, corrected_text) > 0.47

Ну вот, стало ещё немного лучше)